# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Oct 27 2022 9:51AM,249500,10,8695182,Eywa Pharma Inc.,Released
1,Oct 27 2022 9:51AM,249500,10,8695185,Eywa Pharma Inc.,Released
2,Oct 27 2022 9:51AM,249500,10,8695186,Eywa Pharma Inc.,Released
3,Oct 27 2022 9:51AM,249500,10,8695188,Eywa Pharma Inc.,Released
4,Oct 27 2022 9:51AM,249500,10,8695189,Eywa Pharma Inc.,Released
5,Oct 27 2022 9:51AM,249500,10,8695190,Eywa Pharma Inc.,Released
6,Oct 27 2022 9:51AM,249500,10,8695191,Eywa Pharma Inc.,Released
7,Oct 27 2022 9:51AM,249500,10,8695193,Eywa Pharma Inc.,Released
8,Oct 27 2022 9:51AM,249500,10,8695194,Eywa Pharma Inc.,Released
9,Oct 27 2022 9:51AM,249500,10,8695195,Eywa Pharma Inc.,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
15,249492,Released,1
16,249495,Released,4
17,249498,Released,1
18,249499,Released,11
19,249500,Released,12


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
249492,NaN,NaN,1.0
249495,NaN,NaN,4.0
249498,NaN,NaN,1.0
249499,NaN,NaN,11.0
249500,NaN,NaN,12.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248349,0.0,0.0,1.0
248599,0.0,0.0,1.0
249407,0.0,1.0,0.0
249418,0.0,0.0,1.0
249447,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
248349,0,0,1
248599,0,0,1
249407,0,1,0
249418,0,0,1
249447,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248349,0,0,1
1,248599,0,0,1
2,249407,0,1,0
3,249418,0,0,1
4,249447,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,248349,,,1
1,248599,,,1
2,249407,,1,
3,249418,,,1
4,249447,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Oct 27 2022 9:51AM,249500,10,Eywa Pharma Inc.
12,Oct 27 2022 9:47AM,249499,10,Bio-PRF
23,Oct 27 2022 9:37AM,249498,19,"Emersa Waterbox, LLC"
24,Oct 27 2022 9:36AM,249495,10,"Citieffe, Inc."
28,Oct 27 2022 9:08AM,249492,12,SugarBear
29,Oct 27 2022 8:43AM,249488,10,ISDIN Corporation
43,Oct 27 2022 8:35AM,249487,19,"Innogenix, LLC"
44,Oct 27 2022 8:34AM,249485,20,Reliable 1 Laboratories LLC
45,Oct 27 2022 8:21AM,249479,16,Sartorius Bioprocess Solutions
46,Oct 27 2022 7:43AM,249473,19,"AdvaGen Pharma, Ltd"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Oct 27 2022 9:51AM,249500,10,Eywa Pharma Inc.,,,12
1,Oct 27 2022 9:47AM,249499,10,Bio-PRF,,,11
2,Oct 27 2022 9:37AM,249498,19,"Emersa Waterbox, LLC",,,1
3,Oct 27 2022 9:36AM,249495,10,"Citieffe, Inc.",,,4
4,Oct 27 2022 9:08AM,249492,12,SugarBear,,,1
5,Oct 27 2022 8:43AM,249488,10,ISDIN Corporation,1,7,6
6,Oct 27 2022 8:35AM,249487,19,"Innogenix, LLC",,,1
7,Oct 27 2022 8:34AM,249485,20,Reliable 1 Laboratories LLC,,1,
8,Oct 27 2022 8:21AM,249479,16,Sartorius Bioprocess Solutions,,1,
9,Oct 27 2022 7:43AM,249473,19,"AdvaGen Pharma, Ltd",,23,2


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 9:51AM,249500,10,Eywa Pharma Inc.,12,,
1,Oct 27 2022 9:47AM,249499,10,Bio-PRF,11,,
2,Oct 27 2022 9:37AM,249498,19,"Emersa Waterbox, LLC",1,,
3,Oct 27 2022 9:36AM,249495,10,"Citieffe, Inc.",4,,
4,Oct 27 2022 9:08AM,249492,12,SugarBear,1,,
5,Oct 27 2022 8:43AM,249488,10,ISDIN Corporation,6,7,1
6,Oct 27 2022 8:35AM,249487,19,"Innogenix, LLC",1,,
7,Oct 27 2022 8:34AM,249485,20,Reliable 1 Laboratories LLC,,1,
8,Oct 27 2022 8:21AM,249479,16,Sartorius Bioprocess Solutions,,1,
9,Oct 27 2022 7:43AM,249473,19,"AdvaGen Pharma, Ltd",2,23,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 9:51AM,249500,10,Eywa Pharma Inc.,12,,
1,Oct 27 2022 9:47AM,249499,10,Bio-PRF,11,,
2,Oct 27 2022 9:37AM,249498,19,"Emersa Waterbox, LLC",1,,
3,Oct 27 2022 9:36AM,249495,10,"Citieffe, Inc.",4,,
4,Oct 27 2022 9:08AM,249492,12,SugarBear,1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 9:51AM,249500,10,Eywa Pharma Inc.,12.0,NaN,NaN
1,Oct 27 2022 9:47AM,249499,10,Bio-PRF,11.0,NaN,NaN
2,Oct 27 2022 9:37AM,249498,19,"Emersa Waterbox, LLC",1.0,NaN,NaN
3,Oct 27 2022 9:36AM,249495,10,"Citieffe, Inc.",4.0,NaN,NaN
4,Oct 27 2022 9:08AM,249492,12,SugarBear,1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Oct 27 2022 9:51AM,249500,10,Eywa Pharma Inc.,12.0,0.0,0.0
1,Oct 27 2022 9:47AM,249499,10,Bio-PRF,11.0,0.0,0.0
2,Oct 27 2022 9:37AM,249498,19,"Emersa Waterbox, LLC",1.0,0.0,0.0
3,Oct 27 2022 9:36AM,249495,10,"Citieffe, Inc.",4.0,0.0,0.0
4,Oct 27 2022 9:08AM,249492,12,SugarBear,1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,997982,33.0,7.0,1.0
12,748369,3.0,0.0,0.0
16,249479,0.0,1.0,0.0
19,997905,5.0,23.0,0.0
20,249485,0.0,1.0,0.0
21,995806,3.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,997982,33.0,7.0,1.0
1,12,748369,3.0,0.0,0.0
2,16,249479,0.0,1.0,0.0
3,19,997905,5.0,23.0,0.0
4,20,249485,0.0,1.0,0.0
5,21,995806,3.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,33.0,7.0,1.0
1,12,3.0,0.0,0.0
2,16,0.0,1.0,0.0
3,19,5.0,23.0,0.0
4,20,0.0,1.0,0.0
5,21,3.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,33.0
1,12,Released,3.0
2,16,Released,0.0
3,19,Released,5.0
4,20,Released,0.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,19,20,21
Status,,,,,,
Completed,1.0,0.0,0.0,0.0,0.0,0.0
Executing,7.0,0.0,1.0,23.0,1.0,1.0
Released,33.0,3.0,0.0,5.0,0.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,7.0,0.0,1.0,23.0,1.0,1.0
2,Released,33.0,3.0,0.0,5.0,0.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,19,20,21
0,Completed,1.0,0.0,0.0,0.0,0.0,0.0
1,Executing,7.0,0.0,1.0,23.0,1.0,1.0
2,Released,33.0,3.0,0.0,5.0,0.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()